In [14]:
import pandas as pd
import numpy as np

In [16]:
dataset = 'melanoma'
data_folder = 'GSE115978'
info_dir = './patient_info_files/'+dataset+'/'

df = pd.read_csv(info_dir+dataset+'_patients.tsv', '\t')

In [18]:
df.head()

,cells,Patient,1,2,celltype,malignant
0,Cy72_CD45_H02_S758_comb,72.0,1.0,2.0,B,no
1,CY58_1_CD45_B02_S974_comb,58.0,1.0,1.0,T,no
2,Cy71_CD45_D08_S524_comb,71.0,2.0,0.0,Mal,yes
3,Cy81_FNA_CD45_B01_S301_comb,81.0,2.0,0.0,Mal,yes
4,Cy80_II_CD45_B07_S883_comb,80.0,2.0,0.0,Mal,yes


In [19]:
clist = list(pd.read_csv('path_to_data_folder'+data_folder+'/celllist.tsv', header=None).iloc[:,0])

In [20]:
#Only keep cells that have patient info
df = df.loc[df['cells'].isin(clist)]
print(df.shape)

(4097, 6)


In [21]:
df.head()

,cells,Patient,1,2,celltype,malignant
0,Cy72_CD45_H02_S758_comb,72.0,1.0,2.0,B,no
1,CY58_1_CD45_B02_S974_comb,58.0,1.0,1.0,T,no
2,Cy71_CD45_D08_S524_comb,71.0,2.0,0.0,Mal,yes
3,Cy81_FNA_CD45_B01_S301_comb,81.0,2.0,0.0,Mal,yes
4,Cy80_II_CD45_B07_S883_comb,80.0,2.0,0.0,Mal,yes


In [22]:
patient_list = list(set(df['Patient']))
pNum = len(patient_list)
print(pNum)
ind = np.arange(pNum)

19


In [23]:
from sklearn.model_selection import KFold

n_fold = 5

kf = KFold(n_splits=n_fold, shuffle=True)

In [24]:
CV_data = {}
CV_data['test_ind'] = {}
CV_data['train_ind'] = {}
CV_data['nfolds'] = n_fold
# CV_data['col']

In [25]:
j = 0
for train_index, test_index in kf.split(ind):
    patient_train = [patient_list[i] for i in train_index]
    patient_test = [patient_list[i] for i in test_index]
    print(patient_test)
    
    #cell may not be in the list we used in the experiments
    cell_train = df.loc[df['Patient'].isin(patient_train),'cells']
    cell_test = df.loc[df['Patient'].isin(patient_test),'cells']
    
    # Add 1 to the index so that the resulted index is consistant to the one generated by R CV script 
    CV_data['train_ind'][j] = [clist.index(item) for item in list(cell_train.values)]+1
    CV_data['test_ind'][j] = [clist.index(item) for item in list(cell_test.values)]+1
    
    j+=1

[53.0, 60.0, 84.0, 89.0]
[58.0, 67.0, 71.0, 75.0]
[78.0, 80.0, 82.0, 88.0]
[59.0, 72.0, 79.0, 81.0]
[65.0, 74.0, 94.0]


In [26]:
import pickle

pickle.dump(CV_data, open("./"+data_folder+"/CV_data.pkl", "wb"))